# Project - Search Engine

### H συνάρτηση Crawl

In [15]:
import requests
from bs4 import BeautifulSoup

def crawl_wikipedia(url):
    data = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Εξαγωγή τίτλου και κειμένου
        title = soup.find("h1").text
        paragraphs = [p.text for p in soup.find_all("p")]
        content = "\n".join(paragraphs)
        data.append({'title': title, 'content': content})
        
        # Συλλογή τίτλων και περιγραφών
        # for item in soup.select('.mw-parser-output > p'):
        #     title = soup.title.string
        #     description = item.get_text(strip=True)
        #     if description:
        #         data.append({'title': title, 'description': description})
    else: 
        print(f'Error: {response.status_code}')
    
    return data

### Χρηση της συνάρτησης Crawl

In [16]:
# Λίστα άρθρων για συλλογή 
articles = ["Science", "Technology", "Engineering", "Computer"]
collected_data = []

for article in articles:
    url = 'https://en.wikipedia.org/wiki/' + article
    collected_data.extend(crawl_wikipedia(url))

### Αποθήκευση σε JSON

In [17]:
import json
with open('Files/wikipedia_data.json', 'w', encoding='utf-8') as f:
    json.dump(collected_data, f, ensure_ascii=False, indent=4)

### Αποθήκευση σε CSV 

In [18]:
import csv
with open('Files/wikipedia_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['title', 'content'])
    writer.writeheader()
    writer.writerows(collected_data)